In [1]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
import swanlab
import os
from Qtabularfunctions import*
from Cartpolefamily import*

In [2]:
def train_on_random_cartpole(category="medium", episodes=200, bins=[12, 12, 12, 12]):
    gen = CartPoleCategoryGenerator()
    env = gen.generate_env(category)

    agent = TabularQLearningAgent(bins=bins)

    rewards = []

    for ep in range(episodes):
        state, _ = env.reset()
        total_reward = 0

        while True:
            action = agent.choose_action(state)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            agent.update(state, action, reward, next_state, done)

            state = next_state
            total_reward += reward

            if done:
                break

        rewards.append(total_reward)

        if (ep + 1) % 20 == 0:
            print(f"Episode {ep+1}, reward = {total_reward}")

    return agent, rewards


In [3]:
agent, rewards = train_on_random_cartpole(
    category="hard",
    episodes=300
)

Episode 20, reward = 14.0
Episode 40, reward = 10.0
Episode 60, reward = 9.0
Episode 80, reward = 10.0
Episode 100, reward = 10.0
Episode 120, reward = 10.0
Episode 140, reward = 11.0
Episode 160, reward = 11.0
Episode 180, reward = 11.0
Episode 200, reward = 10.0
Episode 220, reward = 17.0
Episode 240, reward = 10.0
Episode 260, reward = 9.0
Episode 280, reward = 12.0
Episode 300, reward = 16.0


In [3]:
# 设置随机数种子
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
# 训练过程
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
agent = DQNAgent(state_dim, action_dim)


# 初始化SwanLab日志记录器
swanlab.init(
    project="RL-All-In-One",
    experiment_name="DQN-CartPole-v1",
    config={
        "state_dim": state_dim,
        "action_dim": action_dim,
        "batch_size": agent.batch_size,
        "gamma": agent.gamma,
        "epsilon": agent.epsilon,
        "update_target_freq": agent.update_target_freq,
        "replay_buffer_size": agent.replay_buffer.maxlen,
        "learning_rate": agent.optimizer.param_groups[0]['lr'],
        "episode": 600,
        "epsilon_start": 1.0,
        "epsilon_end": 0.01,
        "epsilon_decay": 0.995,
    },
    description="增加了初始化目标网络和当前网络一致，避免网络不一致导致的训练波动"
)

# ========== 训练阶段 ==========

agent.epsilon = swanlab.config["epsilon_start"]

for episode in range(swanlab.config["episode"]):
    state = env.reset()[0]
    total_reward = 0

    while True:
        action = agent.choose_action(state)
        next_state, reward, done, _, _ = env.step(action)
        agent.store_experience(state, action, reward, next_state, done)
        agent.train()

        total_reward += reward
        state = next_state
        if done or total_reward > 2e4:
            break

    # epsilon是探索系数，随着每一轮训练，epsilon 逐渐减小
    agent.epsilon = max(swanlab.config["epsilon_end"], agent.epsilon * swanlab.config["epsilon_decay"])  

    # 每10个episode评估一次模型
    if episode % 10 == 0:
        eval_env = gym.make('CartPole-v1')
        avg_reward = agent.evaluate(eval_env)
        eval_env.close()

        if avg_reward > agent.best_avg_reward:
            agent.best_avg_reward = avg_reward
            # 深拷贝当前最优模型的参数
            agent.best_net.load_state_dict({k: v.clone() for k, v in agent.q_net.state_dict().items()})
            agent.save_model(path=f"./output/best_model.pth")
            print(f"New best model saved with average reward: {avg_reward}")

    print(f"Episode: {episode}, Train Reward: {total_reward}, Best Eval Avg Reward: {agent.best_avg_reward}")

    swanlab.log(
        {
            "train/reward": total_reward,
            "eval/best_avg_reward": agent.best_avg_reward,
            "train/epsilon": agent.epsilon
        },
        step=episode,
    )

# 测试并录制视频
agent.epsilon = 0  # 关闭探索策略
test_env = gym.make('CartPole-v1', render_mode='rgb_array')
# test_env = RecordVideo(test_env, "./dqn_videos", episode_trigger=lambda x: True)  # 保存所有测试回合
agent.q_net.load_state_dict(agent.best_net.state_dict())  # 使用最佳模型

for episode in range(3):  # 录制3个测试回合
    state = test_env.reset()[0]
    total_reward = 0
    steps = 0

    while True:
        action = agent.choose_action(state)
        next_state, reward, done, _, _ = test_env.step(action)
        total_reward += reward
        state = next_state
        steps += 1

        # 限制每个episode最多1500步,约30秒,防止录制时间过长
        if done or steps >= 1500:
            break

    print(f"Test Episode: {episode}, Reward: {total_reward}")

test_env.close()


/usr/local/python/3.12.1/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

swanlab: Tracking run with swanlab version 0.7.1

swanlab: Run data will be saved locally in /workspaces/MAB/swanlog/run-20251114_161937-ravrjp8izg6e05sg4vk75

swanlab: 👋 Hi Quan1,welcome to swanlab!

swanlab: Syncing run DQN-CartPole-v1 to the cloud

swanlab: 🏠 View project at https://swanlab.cn/@Quan1/RL-All-In-One

swanlab: 🚀 View run at https://swanlab.cn/@Quan1/RL-All-In-One/runs/ravrjp8izg6e05sg4vk75

Model saved to ./output/best_model.pth
New best model saved with average reward: 9.4
Episode: 0, Train Reward: 19.0, Best Eval Avg Reward: 9.4
Episode: 1, Train Reward: 16.0, Best Eval Avg Reward: 9.4
Episode: 2, Train Reward: 15.0, Best Eval Avg Reward: 9.4
Episode: 3, Train Reward: 31.0, Best Eval Avg Reward: 9.4
Episode: 4, Train Reward: 24.0, Best Eval Avg Reward: 9.4
Episode: 5, Train Reward: 21.0, Best Eval Avg Reward: 9.4
Episode: 6, Train Reward: 13.0, Best Eval Avg Reward: 9.4
Episode: 7, Train Reward: 18.0, Best Eval Avg Reward: 9.4
Episode: 8, Train Reward: 10.0, Best Eval Avg Reward: 9.4
Episode: 9, Train Reward: 15.0, Best Eval Avg Reward: 9.4
Model saved to ./output/best_model.pth
New best model saved with average reward: 14.2
Episode: 10, Train Reward: 18.0, Best Eval Avg Reward: 14.2
Episode: 11, Train Reward: 12.0, Best Eval Avg Reward: 14.2
Episode: 12, Train Reward: 33.0, Best Eval Avg Reward: 14.2
Episode: 13, Train Reward: 17.0, Best Eval Avg Reward: 14.2
Episode: 